In [5]:
pip install surprise

Processing c:\users\jeehy\appdata\local\pip\cache\wheels\76\44\74\b498c42be47b2406bd27994e16c5188e337c657025ab400c1c\scikit_surprise-1.1.1-cp37-cp37m-win_amd64.whl



In [6]:
import surprise

In [7]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [8]:
data=Dataset.load_builtin('ml-100k')

trainset,testset=train_test_split(data,test_size=.25,random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\jeehy/.surprise_data/ml-100k


In [9]:
algo=SVD()

In [10]:
algo.fit(trainset)

In [11]:
predictions=algo.test(testset)
print('prediction type:',type(predictions), 'size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]


prediction type: <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.8441972964221756, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.829175940580603, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.267834354550814, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5942589155640623, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.401342588321873, details={'was_impossible': False})]

In [12]:
[(pred.uid, pred.iid,pred.est) for pred in predictions[:3]]

[('120', '282', 3.8441972964221756),
 ('882', '291', 3.829175940580603),
 ('535', '507', 4.267834354550814)]

In [13]:
uid=str(196)
iid=str(302)
pred=algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.28   {'was_impossible': False}


In [14]:
accuracy.rmse(predictions)

RMSE: 0.9490


0.9490039104199329

In [15]:
import pandas as pd

In [17]:
ratings=pd.read_csv("ratings.csv")
ratings.to_csv("ratings_noh.csv",index=False,header=False)# 헤더 삭제

In [18]:
from surprise import Reader

reader=Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data=Dataset.load_from_file('ratings_noh.csv',reader=reader)

In [19]:
trainset,testset=train_test_split(data,test_size=.25,random_state=0)

algo=SVD(n_factors=50,random_state=0) # 수행시 마다 동일한 결과를 도출하기 위해 random_state 설정

In [20]:
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [22]:
import pandas as pd
from surprise import Reader,Dataset

ratings=pd.read_csv("ratings.csv")
reader=Reader(rating_scale=(0.5,5.0))

data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trainset,testset=train_test_split(data,test_size=.25,random_state=0)

algo=SVD(n_factors=50,random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [23]:
from surprise.model_selection import cross_validate

ratings=pd.read_csv('ratings.csv')
reader=Reader(rating_scale=(0.5,5.0))
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo=SVD(random_state=0)
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8733  0.8661  0.8726  0.8727  0.8702  0.8710  0.0027  
MAE (testset)     0.6707  0.6663  0.6713  0.6695  0.6695  0.6694  0.0017  
Fit time          4.14    4.28    4.50    4.58    4.74    4.45    0.21    
Test time         0.11    0.11    0.18    0.18    0.12    0.14    0.03    


{'test_rmse': array([0.87331545, 0.86609577, 0.87261293, 0.87273436, 0.87016743]),
 'test_mae': array([0.67065004, 0.66634087, 0.67129418, 0.66945129, 0.6694801 ]),
 'fit_time': (4.140182018280029,
  4.284335374832153,
  4.499764442443848,
  4.581191062927246,
  4.738397121429443),
 'test_time': (0.11469340324401855,
  0.11365199089050293,
  0.1825103759765625,
  0.17516231536865234,
  0.11669182777404785)}

In [26]:
from surprise.model_selection import GridSearchCV

param_grid={'n_epochs':[20,40,60],'n_factors':[50,100,200]}
gs=GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


0.8778873563769188
{'n_epochs': 20, 'n_factors': 50}


In [27]:
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
algo=SVD(n_factors=50,random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

Surprise를 이용한 개인화 영화 추천 시스템 구축

In [29]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data_folds=DatasetAutoFolds(ratings_file='ratings_noh.csv',reader=reader)

trainset=data_folds.build_full_trainset()

In [30]:
algo=SVD(n_epochs=20,n_factors=50,random_state=0)
algo.fit(trainset)

In [31]:
movies=pd.read_csv('movies.csv')
movieIds=ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count()==0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [32]:
uid=str(9)
iid=str(42)

pred=algo.predict(uid,iid,verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [33]:
def get_unseen_surprise(ratings,movies,userId):
    seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()
    total_movies=movies['movieId'].tolist()
    
    unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies),'추천 대상 영화 수:',len(unseen_movies),'전체 영화수:',len(total_movies))
    return unseen_movies


unseen_movies=get_unseen_surprise(ratings,movies,9)

평점 매긴 영화수: 46 추천 대상 영화 수: 9696 전체 영화수: 9742


In [38]:
def recomm_movie_by_surprise(algo,userId,unseen_movies,top_n=10):
    predictions=[algo.predict(str(userId),str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est,reverse=True)
    top_predictions=predictions[:top_n]
    
    top_movie_ids=[int(pred.iid) for pred in top_predictions]
    top_movie_rating=[pred.est for pred in top_predictions]
    top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds=[ (id,title,rating) for id, title,rating in \
                     zip(top_movie_ids,top_movie_titles,top_movie_rating)]
    return top_movie_preds

unseen_movies=get_unseen_surprise(ratings,movies,9)
top_movie_preds=recomm_movie_by_surprise(algo,9,unseen_movies,top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1],":",top_movie[2])

평점 매긴 영화수: 46 추천 대상 영화 수: 9696 전체 영화수: 9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
